In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
boston_house = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/Boston_Housing/Training_set_boston.csv")

In [3]:
X = boston_house.iloc[:,:-1]
y = boston_house.MEDV

In [4]:
n_features = X.shape[1]

In [5]:
from sklearn.model_selection import train_test_split 

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=43)

In [7]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.keras import Model

import tensorflow
from numpy.random import seed

In [8]:

seed_value = 42
seed(seed_value)        # If you build the model with given parameters, set_random_seed will help you produce the same result on multiple execution


# Recommended by Keras -------------------------------------------------------------------------------------
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)
# Recommended by Keras -------------------------------------------------------------------------------------

tensorflow.random.set_seed(seed_value)

In [9]:
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import RMSprop

In [10]:
def create_model(optimizers=RMSprop(0.01)):
    inp = Input(shape=(n_features,))
    
    hid = Dense(10,activation="relu")(inp)
    
    hid = Dense(8,activation="relu")(hid)
    
    out = Dense(1)(hid)
    
    model = Model(inputs=inp, outputs= out)
    
    model.compile(loss="mean_squared_error",metrics=['mse'],optimizer=optimizers)
    
    return model

In [11]:
model = KerasRegressor(build_fn=create_model,verbose=1)

In [12]:
batch_size = [10,20,30,40,50]

epochs=[10,20,50]

params_grid = {'batch_size':batch_size,'epochs':epochs}

In [13]:
params_grid

{'batch_size': [10, 20, 30, 40, 50], 'epochs': [10, 20, 50]}

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
grid = GridSearchCV(estimator=model, param_grid=params_grid, cv=5, n_jobs=-1)

# Run the GridSearchCV process
grid_result = grid.fit(X_train, y_train, verbose = 1)

# Print the results of the best model
print('Best params: ' + str(grid_result.best_params_))

Epoch 1/50
17/17 [==============================] - 0s 1ms/step - loss: 2715.2493 - mse: 2715.2493
Epoch 2/50
17/17 [==============================] - 0s 1ms/step - loss: 144.4722 - mse: 144.4722
Epoch 3/50
17/17 [==============================] - 0s 1ms/step - loss: 176.2391 - mse: 176.2391
Epoch 4/50
17/17 [==============================] - 0s 1ms/step - loss: 157.1971 - mse: 157.1971
Epoch 5/50
17/17 [==============================] - 0s 1ms/step - loss: 120.9589 - mse: 120.9589
Epoch 6/50
17/17 [==============================] - 0s 1ms/step - loss: 114.4792 - mse: 114.4792
Epoch 7/50
17/17 [==============================] - 0s 959us/step - loss: 110.8774 - mse: 110.8774
Epoch 8/50
17/17 [==============================] - 0s 977us/step - loss: 106.8383 - mse: 106.8383
Epoch 9/50
17/17 [==============================] - 0s 1ms/step - loss: 78.9711 - mse: 78.9711
Epoch 10/50
17/17 [==============================] - 0s 1ms/step - loss: 66.7577 - mse: 66.7577
Epoch 11/50
17/17 [========

In [16]:
pd.DataFrame(grid_result.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_epochs,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,1.911546,0.070507,0.212224,0.009014,20,50,"{'batch_size': 20, 'epochs': 50}",-28.091606,-50.531643,-25.464216,-41.345612,-49.955063,-39.077628,10.589376,1
2,2.740602,0.107461,0.223810,0.015220,10,50,"{'batch_size': 10, 'epochs': 50}",-62.621315,-60.184845,-27.575737,-26.108664,-40.610634,-43.420239,15.545843,2
4,1.226355,0.045636,0.203558,0.002856,20,20,"{'batch_size': 20, 'epochs': 20}",-33.866211,-79.405014,-32.075813,-49.068954,-51.969337,-49.277066,17.019768,3
10,1.122039,0.154970,0.205726,0.012639,40,20,"{'batch_size': 40, 'epochs': 20}",-42.355965,-74.282387,-40.011410,-57.761349,-58.549248,-54.592072,12.455762,4
8,1.604552,0.068088,0.217889,0.016444,30,50,"{'batch_size': 30, 'epochs': 50}",-29.176754,-68.756592,-33.693027,-45.901535,-123.052231,-60.116028,34.331008,5


In [17]:

# Import the cross validation evaluator
from sklearn.model_selection import cross_val_score

# Measure the model's performance
results = cross_val_score(grid.best_estimator_, X_test, y_test, cv=5)
print('Results: \n  * Mean:', -results.mean(), '\n  * Std:', results.std())

Epoch 1/50
4/4 [==============================] - 0s 2ms/step - loss: 659.9080 - mse: 659.9080
Epoch 2/50
4/4 [==============================] - 0s 2ms/step - loss: 87.0767 - mse: 87.0767
Epoch 3/50
4/4 [==============================] - 0s 2ms/step - loss: 79.4768 - mse: 79.4768
Epoch 4/50
4/4 [==============================] - 0s 2ms/step - loss: 77.8905 - mse: 77.8905
Epoch 5/50
4/4 [==============================] - 0s 2ms/step - loss: 86.5856 - mse: 86.5856
Epoch 6/50
4/4 [==============================] - 0s 2ms/step - loss: 76.4458 - mse: 76.4458
Epoch 7/50
4/4 [==============================] - 0s 3ms/step - loss: 74.9502 - mse: 74.9502
Epoch 8/50
4/4 [==============================] - 0s 2ms/step - loss: 82.7481 - mse: 82.7481
Epoch 9/50
4/4 [==============================] - 0s 2ms/step - loss: 76.2541 - mse: 76.2541
Epoch 10/50
4/4 [==============================] - 0s 2ms/step - loss: 74.4682 - mse: 74.4682
Epoch 11/50
4/4 [==============================] - 0s 2ms/step - lo

In [22]:
import kerastuner as kt

In [19]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 61kB 2.1MB/s 


In [25]:
def model_building(hp):
    inputs = Input(shape=(n_features,))

    hid = Dense(10,activation='relu')(inputs)

    hid_2 = Dense(8,activation='relu')(hid)

    final = Dense(1)(hid_2)

    hp_lr = hp.Choice('learning_rate',values=[0.001,0.01,0.1,1.0])

    optimizer = RMSprop(learning_rate=hp_lr)

    model = Model(inputs=inputs,outputs=final)

    model.compile(loss='mse',metrics=['mse'],optimizer=optimizer)

    return model

In [26]:
tunner = kt.RandomSearch(model_building,objective='mse',seed=42,max_trials=5,directory='random_search')

In [29]:
tunner.search(X_train,y_train,epochs=10,validation_split=0.2,verbose=1)

Epoch 1/10
9/9 [==============================] - 0s 23ms/step - loss: 373.2285 - mse: 373.2285 - val_loss: 152.3379 - val_mse: 152.3379
Epoch 2/10
9/9 [==============================] - 0s 5ms/step - loss: 137.3719 - mse: 137.3719 - val_loss: 98.6123 - val_mse: 98.6123
Epoch 3/10
9/9 [==============================] - 0s 5ms/step - loss: 96.7425 - mse: 96.7425 - val_loss: 85.8221 - val_mse: 85.8221
Epoch 4/10
9/9 [==============================] - 0s 5ms/step - loss: 86.6630 - mse: 86.6630 - val_loss: 86.4883 - val_mse: 86.4883
Epoch 5/10
9/9 [==============================] - 0s 5ms/step - loss: 84.7902 - mse: 84.7902 - val_loss: 80.1648 - val_mse: 80.1648
Epoch 6/10
9/9 [==============================] - 0s 5ms/step - loss: 82.4398 - mse: 82.4398 - val_loss: 85.2267 - val_mse: 85.2267
Epoch 7/10
9/9 [==============================] - 0s 5ms/step - loss: 82.1798 - mse: 82.1798 - val_loss: 73.1422 - val_mse: 73.1422
Epoch 8/10
9/9 [==============================] - 0s 5ms/step - loss:

Epoch 1/10
9/9 [==============================] - 0s 18ms/step - loss: 1397324416.0000 - mse: 1397324416.0000 - val_loss: 6651.0420 - val_mse: 6651.0420
Epoch 2/10
9/9 [==============================] - 0s 5ms/step - loss: 509.1711 - mse: 509.1711 - val_loss: 80.1367 - val_mse: 80.1367
Epoch 3/10
9/9 [==============================] - 0s 5ms/step - loss: 282.7709 - mse: 282.7709 - val_loss: 5511.0752 - val_mse: 5511.0752
Epoch 4/10
9/9 [==============================] - 0s 4ms/step - loss: 1879.9188 - mse: 1879.9188 - val_loss: 633.9631 - val_mse: 633.9631
Epoch 5/10
9/9 [==============================] - 0s 5ms/step - loss: 144.0812 - mse: 144.0812 - val_loss: 74.9064 - val_mse: 74.9064
Epoch 6/10
9/9 [==============================] - 0s 5ms/step - loss: 91.9125 - mse: 91.9125 - val_loss: 947.2366 - val_mse: 947.2366
Epoch 7/10
9/9 [==============================] - 0s 4ms/step - loss: 331.2776 - mse: 331.2776 - val_loss: 327.9899 - val_mse: 327.9899
Epoch 8/10
9/9 [=================

Epoch 1/10
9/9 [==============================] - 0s 18ms/step - loss: 1106.5270 - mse: 1106.5270 - val_loss: 322.5012 - val_mse: 322.5012
Epoch 2/10
9/9 [==============================] - 0s 5ms/step - loss: 188.1669 - mse: 188.1669 - val_loss: 96.6598 - val_mse: 96.6598
Epoch 3/10
9/9 [==============================] - 0s 6ms/step - loss: 89.4503 - mse: 89.4503 - val_loss: 85.8705 - val_mse: 85.8705
Epoch 4/10
9/9 [==============================] - 0s 5ms/step - loss: 84.4604 - mse: 84.4604 - val_loss: 90.4863 - val_mse: 90.4863
Epoch 5/10
9/9 [==============================] - 0s 4ms/step - loss: 84.6403 - mse: 84.6403 - val_loss: 83.6118 - val_mse: 83.6118
Epoch 6/10
9/9 [==============================] - 0s 5ms/step - loss: 84.2324 - mse: 84.2324 - val_loss: 121.2604 - val_mse: 121.2604
Epoch 7/10
9/9 [==============================] - 0s 4ms/step - loss: 90.9207 - mse: 90.9207 - val_loss: 94.4259 - val_mse: 94.4259
Epoch 8/10
9/9 [==============================] - 0s 4ms/step - l

Epoch 1/10
9/9 [==============================] - 0s 20ms/step - loss: 2643.8452 - mse: 2643.8452 - val_loss: 251.7590 - val_mse: 251.7590
Epoch 2/10
9/9 [==============================] - 0s 5ms/step - loss: 129.2674 - mse: 129.2674 - val_loss: 195.2645 - val_mse: 195.2645
Epoch 3/10
9/9 [==============================] - 0s 5ms/step - loss: 102.0941 - mse: 102.0941 - val_loss: 225.6780 - val_mse: 225.6780
Epoch 4/10
9/9 [==============================] - 0s 4ms/step - loss: 134.8096 - mse: 134.8096 - val_loss: 115.7631 - val_mse: 115.7631
Epoch 5/10
9/9 [==============================] - 0s 5ms/step - loss: 77.5195 - mse: 77.5195 - val_loss: 119.3701 - val_mse: 119.3701
Epoch 6/10
9/9 [==============================] - 0s 4ms/step - loss: 100.5684 - mse: 100.5684 - val_loss: 297.9522 - val_mse: 297.9522
Epoch 7/10
9/9 [==============================] - 0s 4ms/step - loss: 90.6295 - mse: 90.6295 - val_loss: 139.0792 - val_mse: 139.0792
Epoch 8/10
9/9 [==============================] -

INFO:tensorflow:Oracle triggered exit


In [30]:
best = tunner.get_best_models()[0]

In [31]:
best.evaluate(X_test,y_test)

3/3 [==============================] - 0s 3ms/step - loss: 103.8843 - mse: 103.8843


[103.88433074951172, 103.88433074951172]

In [32]:
best.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                140       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 237
Trainable params: 237
Non-trainable params: 0
_________________________________________________________________
